In [2]:
%pip install confluent-kafka
%pip install pyspark
%pip install pymongo
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import confluent_kafka as ck
import random
import time
import requests
import csv
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta, date
import calendar
import re
import time

# Kafka

## Config Kafka

In [9]:
config = {}
# Required connection configs for Kafka producer, consumer, and admin
config["bootstrap.servers"] = "pkc-60py3.europe-west9.gcp.confluent.cloud:9092"
#bootstrap.servers=pkc-60py3.europe-west9.gcp.confluent.cloud:9092
config["security.protocol"] = "SASL_SSL"
#security.protocol=SASL_SSL
config["sasl.mechanisms"] = "PLAIN"
#sasl.mechanisms=PLAIN
config["sasl.username"] = "2WQXJOQPDG2MWLJB"
#sasl.username=2WQXJOQPDG2MWLJB
config["sasl.password"] = "N0wpBXjhU3cBEawVm5aX5Ql5IofNTvWf4+CEHQgz+F2sinER5J/p6OJZpFJs0Hd5"
#sasl.password=N0wpBXjhU3cBEawVm5aX5Ql5IofNTvWf4+CEHQgz+F2sinER5J/p6OJZpFJs0Hd5

# Best practice for higher availability in librdkafka clients prior to 1.7
config["session.timeout.ms"] = "45000"
#session.timeout.ms=45000

config["group.id"] = "python-group-1"
config["auto.offset.reset"] = "earliest"
"""

# Required connection configs for Confluent Cloud Schema Registry
schema.registry.url=https://psrc-j55zm.us-central1.gcp.confluent.cloud
basic.auth.credentials.source=USER_INFO
basic.auth.user.info={{ SR_API_KEY }}:{{ SR_API_SECRET }}
"""

'\n\n# Required connection configs for Confluent Cloud Schema Registry\nschema.registry.url=https://psrc-j55zm.us-central1.gcp.confluent.cloud\nbasic.auth.credentials.source=USER_INFO\nbasic.auth.user.info={{ SR_API_KEY }}:{{ SR_API_SECRET }}\n'

# Consumer

In [22]:
from io import StringIO
consumer = ck.Consumer(config)
consumer.subscribe(["mon_topic_demo"])
df = pd.DataFrame(columns=["station_id", "dh_utc", "temperature", "vent_moyen", "pluie_1h"])
try:
     while True:
          msg = consumer.poll(1.0)
          if msg is not None and msg.error() is None:
               input_strings = msg.value().decode('utf-8').split('\n')
               for input_string in input_strings:
                    new_data = pd.read_csv(StringIO(input_string), sep=';', names=df.columns)
                    df = pd.concat([df, new_data], ignore_index=True)
except KeyboardInterrupt:
     pass
finally:
     consumer.close()




C:\Users\Julian\AppData\Local\Temp\ipykernel_11572\1822599183.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


In [23]:
display(df)

,station_id,dh_utc,temperature,vent_moyen,pluie_1h
0,23,2019-11-01 00:00:00,9.2,5.0,0.0
1,23,2019-11-01 01:00:00,9.3,6.0,0.0
2,23,2019-11-01 02:00:00,9.3,5.0,0.0
3,23,2019-11-01 03:00:00,9.3,2.0,0.0
4,23,2019-11-01 04:00:00,9.3,5.0,0.0
...,...,...,...,...,...
17235,ME106,2019-11-30 19:00:00,4.8,11.3,0.0
17236,ME106,2019-11-30 20:00:00,4.7,8.0,0.0
17237,ME106,2019-11-30 21:00:00,4.6,12.9,0.0
17238,ME106,2019-11-30 22:00:00,4.4,11.3,0.0


# Calcul moyennes sur 1 mois

In [24]:
df['dh_utc'] = pd.to_datetime(df['dh_utc'])

# Group by 'station_id' and calculate overall averages
averages_df = df.groupby('station_id').agg({'temperature': 'mean', 'vent_moyen': 'mean', 'pluie_1h': 'mean'}).reset_index()

# Display the resulting DataFrame with overall averages
print(averages_df)

  _____ _        __  __          _   _  ____  _    _ ______   _____  ______  _____    _____ _______    _______ _____ ____  _   _  _____   _   _  ____  _   _    __      ________ _____  _____ ______ _____ ______ _____     _____ _____    _____ ______  _____ _______               _____         _    _  _____ ______   _____  ______  _____   _   _          _   _   _____          _   _  _____   _____  _____   ____  _____  _    _  _____ ______ _____  
 |_   _| |      |  \/  |   /\   | \ | |/ __ \| |  | |  ____| |  __ \|  ____|/ ____|  / ____|__   __|/\|__   __|_   _/ __ \| \ | |/ ____| | \ | |/ __ \| \ | |   \ \    / /  ____|  __ \|_   _|  ____|_   _|  ____|  __ \   / ____|_   _|  / ____|  ____|/ ____|__   __|     /\      / ____|   /\  | |  | |/ ____|  ____| |  __ \|  ____|/ ____| | \ | |   /\   | \ | | |  __ \   /\   | \ | |/ ____| |  __ \|  __ \ / __ \|  __ \| |  | |/ ____|  ____|  __ \ 
   | | | |      | \  / |  /  \  |  \| | |  | | |  | | |__    | |  | | |__  | (___   | (___    | |  /  \  | |    | || |  | |  \| | (___   |  \| | |  | |  \| |    \ \  / /| |__  | |__) | | | | |__    | | | |__  | |__) | | (___   | |   | |    | |__  | (___    | |       /  \    | |       /  \ | |  | | (___ | |__    | |  | | |__  | (___   |  \| |  /  \  |  \| | | |  | | /  \  |  \| | (___   | |__) | |__) | |  | | |  | | |  | | |    | |__  | |__) |
   | | | |      | |\/| | / /\ \ | . ` | |  | | |  | |  __|   | |  | |  __|  \___ \   \___ \   | | / /\ \ | |    | || |  | | . ` |\___ \  | . ` | |  | | . ` |     \ \/ / |  __| |  _  /  | | |  __|   | | |  __| |  _  /   \___ \  | |   | |    |  __|  \___ \   | |      / /\ \   | |      / /\ \| |  | |\___ \|  __|   | |  | |  __|  \___ \  | . ` | / /\ \ | . ` | | |  | |/ /\ \ | . ` |\___ \  |  ___/|  _  /| |  | | |  | | |  | | |    |  __| |  _  / 
  _| |_| |____  | |  | |/ ____ \| |\  | |__| | |__| | |____  | |__| | |____ ____) |  ____) |  | |/ ____ \| |   _| || |__| | |\  |____) | | |\  | |__| | |\  |_     \  /  | |____| | \ \ _| |_| |     _| |_| |____| | \ \   ____) |_| |_  | |____| |____ ____) |  | |     / ____ \  | |____ / ____ \ |__| |____) | |____  | |__| | |____ ____) | | |\  |/ ____ \| |\  | | |__| / ____ \| |\  |____) | | |    | | \ \| |__| | |__| | |__| | |____| |____| | \ \ 
 |_____|______| |_|  |_/_/    \_\_| \_|\___\_\\____/|______| |_____/|______|_____/  |_____/   |_/_/    \_\_|  |_____\____/|_| \_|_____/  |_| \_|\____/|_| \_( )     \/   |______|_|  \_\_____|_|    |_____|______|_|  \_\ |_____/|_____|  \_____|______|_____/   |_|    /_/    \_\  \_____/_/    \_\____/|_____/|______| |_____/|______|_____/  |_| \_/_/    \_\_| \_| |_____/_/    \_\_| \_|_____/  |_|    |_|  \_\\____/|_____/ \____/ \_____|______|_|  \_\
                                                                                                                                                            |/                                                                                                                                                                                                                                                                                                
                                                                                                                                                                                                                                                                                                                                                                                                                                                              

   station_id  temperature  vent_moyen  pluie_1h
0          23     4.643533    1.755216  0.625591
1          42     7.342816    4.994253  0.187931
2       000BZ     8.777260    3.657062  0.231921
3       000C5     7.828969    1.733148  0.182730
4       000CJ     2.894101    3.946489  0.225843
5       000DF    10.066239   10.323504  0.408832
6       000DR     6.914993    4.445686  0.194625
7       000EB     7.086950    7.282270  0.189220
8       000EU     6.970255    6.358782  0.181586
9       000FA     6.191513    5.912447  0.077511
10      000FN     7.653475    5.355603  0.404539
11      000GQ     7.267771    4.654518  0.110843
12      000HL    10.398878    3.447686  0.451052
13      000K0     7.533659    4.986034  0.178492
14      000L7    10.227848    6.113080  0.179466
15      000MJ     7.750927    5.343509  0.233096
16      000OM     8.171049    6.923043  0.145643
17      000OO     9.079743    2.070756  0.455350
18      000Q2     8.134357    4.401170  0.317836
19      000T3     4.